In [1]:
import pandas as pd
import geopandas as gp
import numpy as np

In [2]:
ridehail = pd.read_csv('D:/TNC-Demand-Model/Outputs/2019_Weekday_Monthly_Ridehail_TOD.csv')

In [3]:
len(ridehail)

51328080

In [4]:
agg = {'Trip Seconds':'mean', 'Trip Miles':'mean', 'Fare':'mean', 'Tip':'mean',
       'Additional Charges':'mean', 'Trip Total':'mean', 'PRIVATE_TRIPS':'sum', 'SHARED_TRIPS':'sum',
       'SCALED_SUP_PRIVATE_TRIPS':'sum', 'SCALED_SUP_SHARED_TRIPS':'sum', 'ALL_TRIPS':'sum'}

In [5]:
ridehail_daily = ridehail.groupby(by = ['YEAR','MONTH','ORIGIN','DESTINATION'], as_index = False).agg(agg)

In [6]:
ridehail = pd.DataFrame()

In [7]:
acs = pd.read_csv('D:/TNC-Demand-Model/Outputs/Chicago_ACS_Data.csv')


In [8]:
acs_daily = acs.groupby(by = ['YEAR','MONTH','CENSUS_TRACT'], as_index = False).mean()

In [9]:
acs_daily = acs_daily.drop(['Unnamed: 0','TOD'], axis = 1)

In [10]:
acs = pd.DataFrame()

In [11]:
lehd_od = pd.read_csv('D:/TNC-Demand-Model/Outputs/CHI_LEHD_OD.csv')
lehd_rac = pd.read_csv('D:/TNC-Demand-Model/Outputs/CHI_RAC.csv')
lehd_wac = pd.read_csv('D:/TNC-Demand-Model/Outputs/CHI_WAC.csv')

In [12]:
lehd_od = lehd_od.drop(['Unnamed: 0'], axis = 1)
lehd_rac = lehd_rac.drop(['Unnamed: 0'], axis = 1)
lehd_wac = lehd_wac.drop(['Unnamed: 0'], axis = 1)

In [40]:
otp = pd.read_csv('D:/TNC-Demand-Model/Outputs/OTP_Transit_Travel_Times.csv')

In [41]:
otp_daily = otp.groupby(by = ['YEAR', 'MONTH','origin','destination'],as_index = False).mean()

In [42]:
otp_daily = otp_daily.drop(['Unnamed: 0','year','TOD'], axis = 1)

In [43]:
otp = pd.DataFrame()

In [19]:
len(acs_daily)

21104

In [ ]:
len(lehd_od.TRACTCE10_PICKUP.unique())

In [18]:
est = ridehail_daily.merge(acs_daily, how = 'left', left_on = ['YEAR', 'MONTH', 'ORIGIN'], right_on = ['YEAR', 'MONTH', 'CENSUS_TRACT'], suffixes = ('','_ORIGIN'))
est = est.merge(acs_daily, how = 'left', left_on = ['YEAR', 'MONTH', 'DESTINATION'], right_on = ['YEAR', 'MONTH', 'CENSUS_TRACT'], suffixes = ('','_DESTINATION'))

In [20]:
est['TRACTCE10_PICKUP'] = est.ORIGIN.astype(str)

In [21]:
len(est.TRACTCE10_PICKUP.unique())

801

In [22]:
est['TRACTCE10_PICKUP'] = est['TRACTCE10_PICKUP'].apply(lambda x : x[5:11])
est['TRACTCE10_PICKUP'] = est['TRACTCE10_PICKUP'].astype(int)

In [23]:
est['TRACTCE10_DEST'] = est.DESTINATION.astype(str)

In [24]:
est['TRACTCE10_DEST'] = est['TRACTCE10_DEST'].apply(lambda x : x[5:11])
est['TRACTCE10_DEST'] = est['TRACTCE10_DEST'].astype(int)

In [25]:
est = est.merge(lehd_od, how = 'left', on = ['YEAR', 'MONTH','TRACTCE10_PICKUP', 'TRACTCE10_DEST'], validate = 'one_to_one', suffixes = ('', '_LEHD_OD'))

In [ ]:
est[np.isnan(est['SI01'])]

In [ ]:
lehd_od[(lehd_od['TRACTCE10_PICKUP'] == 10100)&(lehd_od['TRACTCE10_DEST'] == 10400)]

In [150]:
est.SI01.value_counts(dropna = False)

NaN          7879040
0.000000     2025568
1.610770       14215
1.592009       14215
1.611290       14215
              ...   
65.892592          1
44.576247          1
49.352274          1
44.999819          1
43.977737          1
Name: SI01, Length: 466, dtype: int64

In [40]:
lehd_wac.columns

Index(['YEAR', 'MONTH', 'TRACTCE10', 'CNS01', 'CNS02', 'CNS03', 'CNS04',
       'CNS05', 'CNS06', 'CNS07', 'CNS08', 'CNS09', 'CNS10', 'CNS11', 'CNS12',
       'CNS13', 'CNS14', 'CNS15', 'CNS16', 'CNS17', 'CNS18', 'CNS19', 'CNS20'],
      dtype='object')

In [26]:
est['SI01'] = est['SI01'].fillna(0)
est['SI02'] = est['SI02'].fillna(0)
est['SI03'] = est['SI03'].fillna(0)

In [28]:
est = est.merge(lehd_rac, how = 'left', right_on = ['YEAR', 'MONTH','TRACTCE10'], left_on = ['YEAR','MONTH','TRACTCE10_PICKUP'], suffixes = ('', '_LEHD_RAC'))
est = est.merge(lehd_rac, how = 'left', right_on = ['YEAR', 'MONTH','TRACTCE10'], left_on = ['YEAR','MONTH', 'TRACTCE10_DEST'], suffixes = ('_RAC_ORIGIN', '_RAC_DESTINATION'))

est = est.merge(lehd_wac, how = 'left', right_on = ['YEAR', 'MONTH', 'TRACTCE10'], left_on = ['YEAR','MONTH', 'TRACTCE10_PICKUP'], suffixes = ('', '_LEHD_WAC'))
est = est.merge(lehd_wac, how = 'left', right_on = ['YEAR', 'MONTH', 'TRACTCE10'], left_on = ['YEAR','MONTH','TRACTCE10_DEST'], suffixes = ('_WAC_ORIGIN', '_WAC_DESTINATION'))


In [29]:
fill_cols = lehd_wac.columns.drop(['YEAR','MONTH','TRACTCE10'])

In [30]:
for col in fill_cols:
    col = col + '_WAC_ORIGIN'
    est[col] = est[col].fillna(0)

In [31]:
for col in fill_cols:
    col = col + '_WAC_DESTINATION'
    est[col] = est[col].fillna(0)

In [32]:
ridehail_daily = pd.DataFrame()
acs_daily = pd.DataFrame()
acs_daily = pd.DataFrame()
lehd_od = pd.DataFrame()
lehd_rac = pd.DataFrame()
lehd_wac = pd.DataFrame()

In [65]:
otp_daily.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102656160 entries, 0 to 102656159
Data columns (total 6 columns):
 #   Column         Dtype  
---  ------         -----  
 0   origin         int64  
 1   destination    int64  
 2   walk_distance  float64
 3   travel_time    float64
 4   MONTH          int64  
 5   YEAR           int64  
dtypes: float64(2), int64(4)
memory usage: 4.6 GB


In [61]:
est[np.isnan(est['CNS01_WAC_ORIGIN'])]['ORIGIN']

Series([], Name: ORIGIN, dtype: float64)

In [55]:
est.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10265616 entries, 0 to 10265615
Columns: 210 entries, YEAR to CNS20_WAC_DESTINATION
dtypes: float64(202), int32(2), int64(6)
memory usage: 16.1 GB


In [75]:
otp_daily.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102656160 entries, 0 to 102656159
Data columns (total 6 columns):
 #   Column         Dtype  
---  ------         -----  
 0   origin         int64  
 1   destination    int64  
 2   walk_distance  float64
 3   travel_time    float64
 4   MONTH          int64  
 5   YEAR           int64  
dtypes: float64(2), int64(4)
memory usage: 4.6 GB


In [35]:
otp_daily.head()

,origin,destination,walk_distance,travel_time,MONTH,YEAR
0,17031010100,17031010100,46.0,34.0,11,2018
1,17031010100,17031010100,46.0,34.0,11,2018
2,17031010100,17031010100,46.0,34.0,11,2018
3,17031010100,17031010100,46.0,34.0,11,2018
4,17031010100,17031010100,46.0,34.0,11,2018


In [39]:
otp_daily[(otp_daily['origin'] == 17031010100)&(otp_daily['destination'] == 17031010100)&(otp_daily['MONTH'] == 11)]

,origin,destination,walk_distance,travel_time,MONTH,YEAR
0,17031010100,17031010100,46.0,34.0,11,2018
1,17031010100,17031010100,46.0,34.0,11,2018
2,17031010100,17031010100,46.0,34.0,11,2018
3,17031010100,17031010100,46.0,34.0,11,2018
4,17031010100,17031010100,46.0,34.0,11,2018
6416010,17031010100,17031010100,46.0,34.0,11,2019
6416011,17031010100,17031010100,46.0,34.0,11,2019
6416012,17031010100,17031010100,46.0,34.0,11,2019
6416013,17031010100,17031010100,46.0,34.0,11,2019
6416014,17031010100,17031010100,46.0,34.0,11,2019


In [44]:
est = est.merge(otp_daily, how = 'left', right_on = ['YEAR', 'MONTH', 'origin','destination'], left_on = ['YEAR','MONTH','ORIGIN','DESTINATION'], validate = 'one_to_one', suffixes = ('', '_OTP'))

In [45]:
est.head()

,YEAR,MONTH,ORIGIN,DESTINATION,Trip Seconds,Trip Miles,Fare,Tip,Additional Charges,Trip Total,...,CNS15_WAC_DESTINATION,CNS16_WAC_DESTINATION,CNS17_WAC_DESTINATION,CNS18_WAC_DESTINATION,CNS19_WAC_DESTINATION,CNS20_WAC_DESTINATION,origin,destination,walk_distance,travel_time
0,2018,11,1.703101e+10,1.703101e+10,680.883135,2.636944,6.642857,0.647222,2.438071,9.728151,...,0.000000,314.753323,0.00000,0.999367,3.912377,0.0,17031010100,17031010100,46.000000,34.000000
1,2018,11,1.703101e+10,1.703101e+10,320.513095,1.130048,4.161012,0.395714,2.320881,6.877607,...,71.277244,9.167573,0.00000,23.984802,1.956189,0.0,17031010100,17031010201,1395.346021,1116.464363
2,2018,11,1.703101e+10,1.703101e+10,479.683333,1.845000,5.097222,0.416667,2.416667,7.930556,...,44.667073,423.745574,0.00000,29.981003,32.277113,0.0,17031010100,17031010202,806.149782,626.000000
3,2018,11,1.703101e+10,1.703101e+10,208.304545,0.653818,2.933333,0.246667,2.268667,5.448667,...,52.269979,916.757251,2.27544,40.974038,16.627604,0.0,17031010100,17031010300,824.993754,712.017785
4,2018,11,1.703101e+10,1.703101e+10,434.570000,1.592200,5.500000,0.529333,2.050000,8.079333,...,4435.345289,6.111715,3.03392,572.637158,21.518076,0.0,17031010100,17031010400,705.345029,1600.098611


In [46]:
ridehail_daily = pd.DataFrame()
acs = pd.DataFrame()

In [ ]:
YEAR
MONTH
ORIGIN
DESTINATION
Trip Seconds
Trip Miles
Fare
Tip
Additional Charges
Trip Total
PRIVATE_TRIPS
SHARED_TRIPS
SCALED_SUP_PRIVATE_TRIPS
SCALED_SUP_SHARED_TRIPS
ALL_TRIPS
CENSUS_TRACT
TOTAL_POP
TOTAL_MALE
TOTAL_FEMALE
AGE_5U
AGE_5_9
AGE+10_14
AGE_15_19
AGE_20_24
AGE_25_34
AGE_35_44
AGE_45_54
AGE_55_59
AGE_60_64
AGE_65_74
AGE_75_84
AGE_85P
MEDIAN_AGE
TOTAL_HHLDS
AVG_HHLD_SIZE
AVG_FAMILY_SIZE
TOTAL_POP_18_24
TOTAL_POP_18_24_NO_HIGH_SCHOOL
TOTAL_POP_18_24_HIGH_SCHOOL
TOTAL_POP_18_24_SOME_COLLEGE
TOTAL_POP_18_24_SOME_BACHELORS_HIGHER
TOTAL_POP_25P
TOTAL_POP_25P_LESS_THAN_9TH
TOTAL_POP_25P_9TH_TO_12TH
TOTAL_POP_25P_HIGH_SCHOOL
TOTAL_POP_25P_SOME_COLLEGE
TOTAL_POP_25P_ASSOCIATES
TOTAL_POP_25P_BACHELORS
TOTAL_POP_25P_GRADUATE
TOTAL_POP_25P_HIGH_SCHOOL_HIGHER
TOTAL_POP_25P_BACHELORS_HIGHER
TOTAL_HHLDS_LESS_10K
TOTAL_HHLDS_10K_15K
TOTAL_HHLDS_15K_25K
TOTAL_HHLDS_25K_35K
TOTAL_HHLDS_35K_50K
TOTAL_HHLDS_50K_75K
TOTAL_HHLDS_75K_100K
TOTAL_HHLDS_100K_150K
TOTAL_HHLDS_150K_200K
TOTAL_HHLDS_200KP
HHLDS_MEDIAN_INCOME
HHLDS_MEAN_INCOME
HHLDS_0_VEH
HHLDS_1_VEH
HHLDS_2_VEH
HHLDS_3P_VEH
AVG_VEH_PER_HHLD
CENSUS_TRACT_DESTINATION
TOTAL_POP_DESTINATION
TOTAL_MALE_DESTINATION
TOTAL_FEMALE_DESTINATION
AGE_5U_DESTINATION
AGE_5_9_DESTINATION
AGE+10_14_DESTINATION
AGE_15_19_DESTINATION
AGE_20_24_DESTINATION
AGE_25_34_DESTINATION
AGE_35_44_DESTINATION
AGE_45_54_DESTINATION
AGE_55_59_DESTINATION
AGE_60_64_DESTINATION
AGE_65_74_DESTINATION
AGE_75_84_DESTINATION
AGE_85P_DESTINATION
MEDIAN_AGE_DESTINATION
TOTAL_HHLDS_DESTINATION
AVG_HHLD_SIZE_DESTINATION
AVG_FAMILY_SIZE_DESTINATION
TOTAL_POP_18_24_DESTINATION
TOTAL_POP_18_24_NO_HIGH_SCHOOL_DESTINATION
TOTAL_POP_18_24_HIGH_SCHOOL_DESTINATION
TOTAL_POP_18_24_SOME_COLLEGE_DESTINATION
TOTAL_POP_18_24_SOME_BACHELORS_HIGHER_DESTINATION
TOTAL_POP_25P_DESTINATION
TOTAL_POP_25P_LESS_THAN_9TH_DESTINATION
TOTAL_POP_25P_9TH_TO_12TH_DESTINATION
TOTAL_POP_25P_HIGH_SCHOOL_DESTINATION
TOTAL_POP_25P_SOME_COLLEGE_DESTINATION
TOTAL_POP_25P_ASSOCIATES_DESTINATION
TOTAL_POP_25P_BACHELORS_DESTINATION
TOTAL_POP_25P_GRADUATE_DESTINATION
TOTAL_POP_25P_HIGH_SCHOOL_HIGHER_DESTINATION
TOTAL_POP_25P_BACHELORS_HIGHER_DESTINATION
TOTAL_HHLDS_LESS_10K_DESTINATION
TOTAL_HHLDS_10K_15K_DESTINATION
TOTAL_HHLDS_15K_25K_DESTINATION
TOTAL_HHLDS_25K_35K_DESTINATION
TOTAL_HHLDS_35K_50K_DESTINATION
TOTAL_HHLDS_50K_75K_DESTINATION
TOTAL_HHLDS_75K_100K_DESTINATION
TOTAL_HHLDS_100K_150K_DESTINATION
TOTAL_HHLDS_150K_200K_DESTINATION
TOTAL_HHLDS_200KP_DESTINATION
HHLDS_MEDIAN_INCOME_DESTINATION
HHLDS_MEAN_INCOME_DESTINATION
HHLDS_0_VEH_DESTINATION
HHLDS_1_VEH_DESTINATION
HHLDS_2_VEH_DESTINATION
HHLDS_3P_VEH_DESTINATION
AVG_VEH_PER_HHLD_DESTINATION
TRACTCE10_PICKUP
TRACTCE10_DEST
SI01
SI02
SI03
TRACTCE10_RAC_ORIGIN
CNS01_RAC_ORIGIN
CNS02_RAC_ORIGIN
CNS03_RAC_ORIGIN
CNS04_RAC_ORIGIN
CNS05_RAC_ORIGIN
CNS06_RAC_ORIGIN
CNS07_RAC_ORIGIN
CNS08_RAC_ORIGIN
CNS09_RAC_ORIGIN
CNS10_RAC_ORIGIN
CNS11_RAC_ORIGIN
CNS12_RAC_ORIGIN
CNS13_RAC_ORIGIN
CNS14_RAC_ORIGIN
CNS15_RAC_ORIGIN
CNS16_RAC_ORIGIN
CNS17_RAC_ORIGIN
CNS18_RAC_ORIGIN
CNS19_RAC_ORIGIN
CNS20_RAC_ORIGIN
TRACTCE10_RAC_DESTINATION
CNS01_RAC_DESTINATION
CNS02_RAC_DESTINATION
CNS03_RAC_DESTINATION
CNS04_RAC_DESTINATION
CNS05_RAC_DESTINATION
CNS06_RAC_DESTINATION
CNS07_RAC_DESTINATION
CNS08_RAC_DESTINATION
CNS09_RAC_DESTINATION
CNS10_RAC_DESTINATION
CNS11_RAC_DESTINATION
CNS12_RAC_DESTINATION
CNS13_RAC_DESTINATION
CNS14_RAC_DESTINATION
CNS15_RAC_DESTINATION
CNS16_RAC_DESTINATION
CNS17_RAC_DESTINATION
CNS18_RAC_DESTINATION
CNS19_RAC_DESTINATION
CNS20_RAC_DESTINATION
TRACTCE10_WAC_ORIGIN
CNS01_WAC_ORIGIN
CNS02_WAC_ORIGIN
CNS03_WAC_ORIGIN
CNS04_WAC_ORIGIN
CNS05_WAC_ORIGIN
CNS06_WAC_ORIGIN
CNS07_WAC_ORIGIN
CNS08_WAC_ORIGIN
CNS09_WAC_ORIGIN
CNS10_WAC_ORIGIN
CNS11_WAC_ORIGIN
CNS12_WAC_ORIGIN
CNS13_WAC_ORIGIN
CNS14_WAC_ORIGIN
CNS15_WAC_ORIGIN
CNS16_WAC_ORIGIN
CNS17_WAC_ORIGIN
CNS18_WAC_ORIGIN
CNS19_WAC_ORIGIN
CNS20_WAC_ORIGIN
TRACTCE10_WAC_DESTINATION
CNS01_WAC_DESTINATION
CNS02_WAC_DESTINATION
CNS03_WAC_DESTINATION
CNS04_WAC_DESTINATION
CNS05_WAC_DESTINATION
CNS06_WAC_DESTINATION
CNS07_WAC_DESTINATION
CNS08_WAC_DESTINATION
CNS09_WAC_DESTINATION
CNS10_WAC_DESTINATION
CNS11_WAC_DESTINATION
CNS12_WAC_DESTINATION
CNS13_WAC_DESTINATION
CNS14_WAC_DESTINATION
CNS15_WAC_DESTINATION
CNS16_WAC_DESTINATION
CNS17_WAC_DESTINATION
CNS18_WAC_DESTINATION
CNS19_WAC_DESTINATION
CNS20_WAC_DESTINATION
origin
destination
walk_distance
travel_time

In [53]:
est['TOTAL_JOBS_ORIGIN'] = est['CNS01_WAC_ORIGIN'] +est['CNS02_WAC_ORIGIN']+est['CNS03_WAC_ORIGIN']+est['CNS04_WAC_ORIGIN']+est['CNS05_WAC_ORIGIN']+est['CNS06_WAC_ORIGIN']+est['CNS07_WAC_ORIGIN']+est['CNS08_WAC_ORIGIN']+est['CNS09_WAC_ORIGIN']+est['CNS10_WAC_ORIGIN']+est['CNS11_WAC_ORIGIN']+est['CNS12_WAC_ORIGIN']+est['CNS13_WAC_ORIGIN']+est['CNS14_WAC_ORIGIN']+est['CNS15_WAC_ORIGIN']+est['CNS16_WAC_ORIGIN']+est['CNS17_WAC_ORIGIN']+est['CNS18_WAC_ORIGIN']+est['CNS19_WAC_ORIGIN']+est['CNS20_WAC_ORIGIN']

In [80]:
est['TOTAL_JOBS_DESTINATION'] = est['CNS01_WAC_DESTINATION'] +est['CNS02_WAC_DESTINATION']+est['CNS03_WAC_DESTINATION']+est['CNS04_WAC_DESTINATION']+est['CNS05_WAC_DESTINATION']+est['CNS06_WAC_DESTINATION']+est['CNS07_WAC_DESTINATION']+est['CNS08_WAC_DESTINATION']+est['CNS09_WAC_DESTINATION']+est['CNS10_WAC_DESTINATION']+est['CNS11_WAC_DESTINATION']+est['CNS12_WAC_DESTINATION']+est['CNS13_WAC_DESTINATION']+est['CNS14_WAC_DESTINATION']+est['CNS15_WAC_DESTINATION']+est['CNS16_WAC_DESTINATION']+est['CNS17_WAC_DESTINATION']+est['CNS18_WAC_DESTINATION']+est['CNS19_WAC_DESTINATION']+est['CNS20_WAC_DESTINATION']

In [57]:
est['TOTAL_WORKERS_ORIGIN'] = est['CNS01_RAC_ORIGIN']+est['CNS02_RAC_ORIGIN']+est['CNS03_RAC_ORIGIN']+est['CNS04_RAC_ORIGIN']+est['CNS05_RAC_ORIGIN']+est['CNS06_RAC_ORIGIN']+est['CNS07_RAC_ORIGIN']+est['CNS08_RAC_ORIGIN']+est['CNS09_RAC_ORIGIN']+est['CNS10_RAC_ORIGIN']+est['CNS11_RAC_ORIGIN']+est['CNS12_RAC_ORIGIN']+est['CNS13_RAC_ORIGIN']+est['CNS14_RAC_ORIGIN']+est['CNS15_RAC_ORIGIN']+est['CNS16_RAC_ORIGIN']+est['CNS17_RAC_ORIGIN']+est['CNS18_RAC_ORIGIN']+est['CNS19_RAC_ORIGIN']+est['CNS20_RAC_ORIGIN']

In [58]:
est['TOTAL_WORKERS_DESTINATION'] = est['CNS01_RAC_DESTINATION']+est['CNS02_RAC_DESTINATION']+est['CNS03_RAC_DESTINATION']+est['CNS04_RAC_DESTINATION']+est['CNS05_RAC_DESTINATION']+est['CNS06_RAC_DESTINATION']+est['CNS07_RAC_DESTINATION']+est['CNS08_RAC_DESTINATION']+est['CNS09_RAC_DESTINATION']+est['CNS10_RAC_DESTINATION']+est['CNS11_RAC_DESTINATION']+est['CNS12_RAC_DESTINATION']+est['CNS13_RAC_DESTINATION']+est['CNS14_RAC_DESTINATION']+est['CNS15_RAC_DESTINATION']+est['CNS16_RAC_DESTINATION']+est['CNS17_RAC_DESTINATION']+est['CNS18_RAC_DESTINATION']+est['CNS19_RAC_DESTINATION']+est['CNS20_RAC_DESTINATION']

In [48]:
for col in est.columns:
    print(col)

YEAR
MONTH
ORIGIN
DESTINATION
Trip Seconds
Trip Miles
Fare
Tip
Additional Charges
Trip Total
PRIVATE_TRIPS
SHARED_TRIPS
SCALED_SUP_PRIVATE_TRIPS
SCALED_SUP_SHARED_TRIPS
ALL_TRIPS
CENSUS_TRACT
TOTAL_POP
TOTAL_MALE
TOTAL_FEMALE
AGE_5U
AGE_5_9
AGE+10_14
AGE_15_19
AGE_20_24
AGE_25_34
AGE_35_44
AGE_45_54
AGE_55_59
AGE_60_64
AGE_65_74
AGE_75_84
AGE_85P
MEDIAN_AGE
TOTAL_HHLDS
AVG_HHLD_SIZE
AVG_FAMILY_SIZE
TOTAL_POP_18_24
TOTAL_POP_18_24_NO_HIGH_SCHOOL
TOTAL_POP_18_24_HIGH_SCHOOL
TOTAL_POP_18_24_SOME_COLLEGE
TOTAL_POP_18_24_SOME_BACHELORS_HIGHER
TOTAL_POP_25P
TOTAL_POP_25P_LESS_THAN_9TH
TOTAL_POP_25P_9TH_TO_12TH
TOTAL_POP_25P_HIGH_SCHOOL
TOTAL_POP_25P_SOME_COLLEGE
TOTAL_POP_25P_ASSOCIATES
TOTAL_POP_25P_BACHELORS
TOTAL_POP_25P_GRADUATE
TOTAL_POP_25P_HIGH_SCHOOL_HIGHER
TOTAL_POP_25P_BACHELORS_HIGHER
TOTAL_HHLDS_LESS_10K
TOTAL_HHLDS_10K_15K
TOTAL_HHLDS_15K_25K
TOTAL_HHLDS_25K_35K
TOTAL_HHLDS_35K_50K
TOTAL_HHLDS_50K_75K
TOTAL_HHLDS_75K_100K
TOTAL_HHLDS_100K_150K
TOTAL_HHLDS_150K_200K
TOTAL_HHLDS_2

In [81]:
est.to_csv('D:/TNC-Demand-Model/Outputs/Chi_Estimation_File.csv')

KeyboardInterrupt: 

In [125]:
#rename = ['Pickup Census Tract',
'Dropoff Census Tract',
'MONTH',
'DAY',
'Trip Seconds',
'Trip Miles',
'Fare',
'Tip',
'Additional Charges',
'Trip Total',
'PRIVATE_TRIPS',
'SHARED_TRIPS',
'Trips Pooled',
'TOD',
'TOTAL_POP_ORIGIN',
'TOTAL_MALE_ORIGIN',
'TOTAL_FEMALE_ORIGIN',
'AGE_5U_ORIGIN',
'AGE_5_9_ORIGIN',
'AGE_10_14_ORIGIN',
'AGE_15_19_ORIGIN',
'AGE_20_24_ORIGIN',
'AGE_25_34_ORIGIN',
'AGE_35_44_ORIGIN',
'AGE_45_54_ORIGIN',
'AGE_55_59_ORIGIN',
'AGE_60_64_ORIGIN',
'AGE_65_74_ORIGIN',
'AGE_75_84_ORIGIN',
'AGE_85P_ORIGIN',
'MEDIAN_AGE_ORIGIN',
'TOTAL_HHLDS_ORIGIN',
'AVG_HHLD_SIZE_ORIGIN',
'AVG_FAMILY_SIZE_ORIGIN',
'TOTAL_POP_18_24_ORIGIN',
'TOTAL_POP_18_24_NO_HIGH_SCHOOL_ORIGIN',
'TOTAL_POP_18_24_HIGH_SCHOOL_ORIGIN',
'TOTAL_POP_18_24_SOME_COLLEGE_ORIGIN',
'TOTAL_POP_18_24_SOME_BACHELORS_HIGHER_ORIGIN',
'TOTAL_POP_25P_ORIGIN',
'TOTAL_POP_25P_LESS_THAN_9TH_ORIGIN',
'TOTAL_POP_25P_9TH_TO_12TH_ORIGIN',
'TOTAL_POP_25P_HIGH_SCHOOL_ORIGIN',
'TOTAL_POP_25P_SOME_COLLEGE_ORIGIN',
'TOTAL_POP_25P_ASSOCIATES_ORIGIN',
'TOTAL_POP_25P_BACHELORS_ORIGIN',
'TOTAL_POP_25P_GRADUATE_ORIGIN',
'TOTAL_POP_25P_HIGH_SCHOOL_HIGHER_ORIGIN',
'TOTAL_POP_25P_BACHELORS_HIGHER_ORIGIN',
'TOTAL_HHLDS_LESS_10K_ORIGIN',
'TOTAL_HHLDS_10K_15K_ORIGIN',
'TOTAL_HHLDS_15K_25K_ORIGIN',
'TOTAL_HHLDS_25K_35K_ORIGIN',
'TOTAL_HHLDS_35K_50K_ORIGIN',
'TOTAL_HHLDS_50K_75K_ORIGIN',
'TOTAL_HHLDS_75K_100K_ORIGIN',
'TOTAL_HHLDS_100K_150K_ORIGIN',
'TOTAL_HHLDS_150K_200K_ORIGIN',
'TOTAL_HHLDS_200KP_ORIGIN',
'HHLDS_MEDIAN_INCOME_ORIGIN',
'HHLDS_MEAN_INCOME_ORIGIN',
'TOTAL_POP_DESTINATION',
'TOTAL_MALE_DESTINATION',
'TOTAL_FEMALE_DESTINATION',
'AGE_5U_DESTINATION',
'AGE_5_9_DESTINATION',
'AGE_10_14_DESTINATION',
'AGE_15_19_DESTINATION',
'AGE_20_24_DESTINATION',
'AGE_25_34_DESTINATION',
'AGE_35_44_DESTINATION',
'AGE_45_54_DESTINATION',
'AGE_55_59_DESTINATION',
'AGE_60_64_DESTINATION',
'AGE_65_74_DESTINATION',
'AGE_75_84_DESTINATION',
'AGE_85P_DESTINATION',
'MEDIAN_AGE_DESTINATION',
'CENSUS_TRACT_DESTINATION',
'TOTAL_HHLDS_DESTINATION',
'AVG_HHLD_SIZE_DESTINATION',
'AVG_FAMILY_SIZE_DESTINATION',
'TOTAL_POP_18_24_DESTINATION',
'TOTAL_POP_18_24_NO_HIGH_SCHOOL_DESTINATION',
'TOTAL_POP_18_24_HIGH_SCHOOL_DESTINATION',
'TOTAL_POP_18_24_SOME_COLLEGE_DESTINATION',
'TOTAL_POP_18_24_SOME_BACHELORS_HIGHER_DESTINATION',
'TOTAL_POP_25P_DESTINATION',
'TOTAL_POP_25P_LESS_THAN_9TH_DESTINATION',
'TOTAL_POP_25P_9TH_TO_12TH_DESTINATION',
'TOTAL_POP_25P_HIGH_SCHOOL_DESTINATION',
'TOTAL_POP_25P_SOME_COLLEGE_DESTINATION',
'TOTAL_POP_25P_ASSOCIATES_DESTINATION',
'TOTAL_POP_25P_BACHELORS_DESTINATION',
'TOTAL_POP_25P_GRADUATE_DESTINATION',
'TOTAL_POP_25P_HIGH_SCHOOL_HIGHER_DESTINATION',
'TOTAL_POP_25P_BACHELORS_HIGHER_DESTINATION',
'TOTAL_HHLDS_LESS_10K_DESTINATION',
'TOTAL_HHLDS_10K_15K_DESTINATION',
'TOTAL_HHLDS_15K_25K_DESTINATION',
'TOTAL_HHLDS_25K_35K_DESTINATION',
'TOTAL_HHLDS_35K_50K_DESTINATION',
'TOTAL_HHLDS_50K_75K_DESTINATION',
'TOTAL_HHLDS_75K_100K_DESTINATION',
'TOTAL_HHLDS_100K_150K_DESTINATION',
'TOTAL_HHLDS_150K_200K_DESTINATION',
'TOTAL_HHLDS_200KP_DESTINATION',
'HHLDS_MEDIAN_INCOME_DESTINATION',
'HHLDS_MEAN_INCOME_DESTINATION',
'ORIGIN_CENSUS_TRACT',
'DESTINATION_CENSUS_TRACT',
'TOTAL_EMP_LEHD_OD',
'29U_EMP_LEHD_OD',
'30_54_EMP_LEHD_OD',
'55P_EMP_LEHD_OD',
'1250U_EMP_LEHD_OD',
'1251_3333_EMP_LEHD_OD',
'3333P_EMP_LEHD_OD',
'TOTAL_EMP_RAC_ORIGIN',
'29U_EMP_RAC_ORIGIN',
'30_54_EMP_RAC_ORIGIN',
'55P_EMP_RAC_ORIGIN',
'1250U_EMP_RAC_ORIGIN',
'1251_3333__RAC_ORIGIN',
'3333P_EMP_RAC_ORIGIN',
'FOOD_SERVICE_EMP_RAC_ORIGIN',
'TOTAL_EMP_RAC_DESTINATION',
'29U_EMP_RAC_DESTINATION',
'30_54_EMP_RAC_DESTINATION',
'55P_EMP_RAC_DESTINATION',
'1250U_EMP_RAC_DESTINATION',
'1251_3333_EMP_RAC_DESTINATION',
'3333P_EMP_RAC_DESTINATION',
'FOOD_SERVICE_EMP_RAC_DESTINATION',
'TOTAL_EMP_WAC_ORIGIN',
'29U_EMP_WAC_ORIGIN',
'30_54_EMP_WAC_ORIGIN',
'55P_EMP_WAC_ORIGIN',
'1250U_EMP_WAC_ORIGIN',
'1251_3333__WAC_ORIGIN',
'333P_EMP_WAC_ORIGIN',
'FOOD_SERVICE_EMP_WAC_ORIGIN',
'TOTAL_EMP_WAC_DESTINATION',
'29U_EMP_WAC_DESTINATION',
'30_54_EMP_WAC_DESTINATION',
'55P_EMP_WAC_DESTINATION',
'1250U_EMP_WAC_DESTINATION',
'1251_3333__WAC_DESTINATION',
'3333P_EMP_WAC_DESTINATION',
'FOOD_SERVICE_EMP_WAC_DESTINATION']

In [126]:
est.columns = rename

In [127]:
est.head()

,Pickup Census Tract,Dropoff Census Tract,MONTH,DAY,Trip Seconds,Trip Miles,Fare,Tip,Additional Charges,Trip Total,...,333P_EMP_WAC_ORIGIN,FOOD_SERVICE_EMP_WAC_ORIGIN,TOTAL_EMP_WAC_DESTINATION,29U_EMP_WAC_DESTINATION,30_54_EMP_WAC_DESTINATION,55P_EMP_WAC_DESTINATION,1250U_EMP_WAC_DESTINATION,1251_3333__WAC_DESTINATION,3333P_EMP_WAC_DESTINATION,FOOD_SERVICE_EMP_WAC_DESTINATION
0,1.703101e+10,1.703101e+10,1.545455,14.681818,357.088636,1.089394,4.126894,0.606061,2.351250,7.084205,...,2777.451762,713.972031,9079.511933,1994.093716,4582.493729,2502.924488,2668.847566,3633.212605,2777.451762,713.972031
1,1.703101e+10,1.703101e+10,1.482759,15.310345,285.902299,1.069540,3.735632,0.068966,2.387069,6.191667,...,2777.451762,713.972031,1205.707698,282.572029,629.502101,293.633568,506.819583,519.892310,178.995805,33.184616
2,1.703101e+10,1.703101e+10,1.384615,16.000000,230.307692,0.576923,2.884615,0.076923,2.153846,5.115385,...,2777.451762,713.972031,5710.770657,1572.749658,2903.151063,1234.869936,1616.995812,2210.296514,1883.478331,348.941260
3,1.703101e+10,1.703101e+10,1.500000,14.333333,206.625000,0.700000,3.333333,0.250000,2.125000,5.708333,...,2777.451762,713.972031,6953.685349,1191.629376,3950.980439,1811.075533,1711.521687,2680.914699,2561.248964,203.130071
4,1.703101e+10,1.703101e+10,1.588235,10.764706,365.431373,1.462745,4.950980,0.490196,1.647059,7.088235,...,2777.451762,713.972031,17294.212673,3218.907708,9510.911935,4564.393029,3526.619598,5732.893735,8034.699340,1123.248957


In [64]:
drop_str = ['ORIGIN',
'DESTINATION',
'MONTH',
'CENSUS_TRACT_DESTINATION','TRACTCE10_PICKUP','TRACTCE10_DEST','TRACTCE10_RAC_ORIGIN','TRACTCE10_RAC_DESTINATION','TRACTCE10_WAC_ORIGIN','TRACTCE10_WAC_DESTINATION']


In [ ]:
est_stats = est.drop(drop_str, axis = 1)

In [ ]:
def find_mean(column):
    column = pd.to_numeric(column)
    output = column.mean()
    
    return output

def find_min(column):
    column = pd.to_numeric(column)
    output = column.min()
    
    return output

def find_max(column):
    column = pd.to_numeric(column)
    output = column.max()
    
    return output

In [ ]:
est_stats = est_stats.replace('-', '0')
est_stats = est_stats.replace('250,000+', '250000')

In [ ]:
df = pd.DataFrame()
df['Mean'] = est_stats.apply(lambda column: find_mean(column), axis = 0)
df['Min'] = est_stats.apply(lambda column: find_min(column), axis = 0)
df['Max'] = est_stats.apply(lambda column: find_max(column), axis = 0)

In [ ]:
df.head()

In [ ]:
df.to_csv('D:/TNC-Demand-Model/Outputs/Descriptive Statistics of 2019 Estimation File.csv')

In [ ]:
first_100 = est.head(100)

In [ ]:
first_100.to_csv('D:/TNC-Demand-Model/Outputs/Chi_Estimation_File_2019 (First 100 rows).csv')

In [ ]:
cols = est.columns

In [75]:
df2 = pd.DataFrame()
df2['VARIABLE'] = cols

In [79]:
df2.to_csv('D:/TNC-Demand-Model/Outputs/Data Dictionary for Chi Estimation File.csv')